# Steps to Calculate the Entropy of Parkinsonian Rest Tremors

Assuming imported files have the columns Time,AccelX,AccelY,AccelZ,RotateX,RotateY,RotateZ only

from bokeh.io import output_notebook, show
from bokeh.plotting import figure

plot = figure(plot_width=400, tools='pan,box_zoom')
#plot.circle([1,2,3,4,5],[8,6,5,2,3])
plot.circle(x=10,y=[2,5,8,12],size=[10,20,30,40])
output_notebook()
show(plot)

## 0. Import Relevant Libraries

In [73]:
import pandas as pd
from scipy.stats import zscore # normalize data. Not used.
import scipy.signal as signal
import matplotlib.pyplot as plt
import numpy as np
import sampen
import glob
import os

from sampen import sampen2
    # https://github.com/bergantine/sampen/
from pyentrp import entropy as ent
    # https://github.com/nikdon/pyEntropy 
    
import sys
sys.path.insert(0, 'C:\\Users\\albei\\OneDrive\\Desktop\\entropy\\entropy\\')
from entropy import *
    # https://github.com/raphaelvallat/entropy

## 1. Rename and Convert Files

### Choose Global Variables

In [63]:
prepost = "post"  # pre or post
entropy = "samen" # samen or appen

In [64]:
path = "input\\" + prepost
all_csv = glob.glob(os.path.join(path, "*.csv"))  # make a list of paths
shape = len(all_csv)

In [65]:
def convert(all_csv):
    ''' Converts csv to xlsx
    
    Converts and renames files. Filename must be in format of "DDMMYYY_####_[identifier_here].csv"
    '''
    i = 0
    for file in all_csv:
        csv_file = file.split(sep='\\')[2][14:-4] # get file's name without extension
        dir_path = os.path.dirname(os.path.realpath(file)) # get the csv file's current directory
        xlsx_file = dir_path + "\\"+ csv_file + ".xlsx" # save csv file in current directory
        pd.read_csv(file, delimiter= ",").to_excel(xlsx_file, index=False)
        i = i + 1
        print("||",i, "/", shape, " FINISHED! || ", csv_file)
    print("All files converted and saved in ", path)

In [17]:
convert(all_csv)

|| 1 / 23  FINISHED! ||  BDNF011_RestTremor
|| 2 / 23  FINISHED! ||  BDNF010_RestTremor
|| 3 / 23  FINISHED! ||  BDNF022_RestTremor
|| 4 / 23  FINISHED! ||  BDNF005_RestTremor
|| 5 / 23  FINISHED! ||  BDNF015_RestTremor
|| 6 / 23  FINISHED! ||  BDNF024_RestTremor
|| 7 / 23  FINISHED! ||  BDNF014_RestTremor
|| 8 / 23  FINISHED! ||  BDNF023_RestTremor
|| 9 / 23  FINISHED! ||  BDNF016_RestTremor
|| 10 / 23  FINISHED! ||  BDNF018_RestTremor
|| 11 / 23  FINISHED! ||  BDNF021_RestTremor
|| 12 / 23  FINISHED! ||  BDNF012_RestTremor
|| 13 / 23  FINISHED! ||  BDNF017_RestTremor
|| 14 / 23  FINISHED! ||  BDNF006_RestTremor
|| 15 / 23  FINISHED! ||  BDNF009_RestTremor
|| 16 / 23  FINISHED! ||  BDNF013_RestTremor
|| 17 / 23  FINISHED! ||  BDNF007_RestTremor
|| 18 / 23  FINISHED! ||  BDNF004_RestTremor
|| 19 / 23  FINISHED! ||  BDNF020_RestTremor
|| 20 / 23  FINISHED! ||  BDNF003_RestTremor
|| 21 / 23  FINISHED! ||  BDNF008_RestTremor
|| 22 / 23  FINISHED! ||  BDNF002_RestTremor
|| 23 / 23  FINISHE

## 2. Butterworth Filtering

In [66]:
all_xlsx = glob.glob(os.path.join(path, "*.xlsx"))  # make a list of paths
shape = len(all_xlsx)

In [67]:
def butter_filter(all_xlsx):
    ''' Butters up raw data through 4th order filter with 30hz cutoff frequency
    
    Pass all data through butter filter. Paper by Gil, et al (2010) said they passed it 
    through 4th order butter filter with 30hz cutoff frequency
    '''
    i = 0
    for file in all_xlsx:
        # Setup
        i = i+1
        df = pd.read_excel(file)
        cols = df.columns[1:] # all columns except time
        butter_dic = {}
        butter_dic['Time'] = df['Time']
        
        # Design the Buterworth filter
        N  = 4    # Filter order
        Wn = 0.3 # Cutoff frequency
        B, A = signal.butter(N, Wn, output='ba')

        # Second, apply the filter
        for col in cols:    
            tremor = np.array(df[col]) # butter only takes arrays
            tremor_f = signal.filtfilt(B,A, tremor)
            butter_dic[col] = list(tremor_f) # turn it back to list then add to dictionary
        new_file = file.split(sep='.')[0] + '_butter.xlsx' # rename file
        butter_df = pd.DataFrame.from_dict(butter_dic).reset_index()
        butter_df.to_excel(new_file)
        print("||", i, '/', shape, " FINISHED! || ", new_file.split(sep='\\')[2])
    
    print("Finished! All butter saved to", path)

In [20]:
butter_filter(all_xlsx)

|| 1 / 23  FINISHED! ||  BDNF001_RestTremor_butter.xlsx
|| 2 / 23  FINISHED! ||  BDNF002_RestTremor_butter.xlsx
|| 3 / 23  FINISHED! ||  BDNF003_RestTremor_butter.xlsx
|| 4 / 23  FINISHED! ||  BDNF004_RestTremor_butter.xlsx
|| 5 / 23  FINISHED! ||  BDNF005_RestTremor_butter.xlsx
|| 6 / 23  FINISHED! ||  BDNF006_RestTremor_butter.xlsx
|| 7 / 23  FINISHED! ||  BDNF007_RestTremor_butter.xlsx
|| 8 / 23  FINISHED! ||  BDNF008_RestTremor_butter.xlsx
|| 9 / 23  FINISHED! ||  BDNF009_RestTremor_butter.xlsx
|| 10 / 23  FINISHED! ||  BDNF010_RestTremor_butter.xlsx
|| 11 / 23  FINISHED! ||  BDNF011_RestTremor_butter.xlsx
|| 12 / 23  FINISHED! ||  BDNF012_RestTremor_butter.xlsx
|| 13 / 23  FINISHED! ||  BDNF013_RestTremor_butter.xlsx
|| 14 / 23  FINISHED! ||  BDNF014_RestTremor_butter.xlsx
|| 15 / 23  FINISHED! ||  BDNF015_RestTremor_butter.xlsx
|| 16 / 23  FINISHED! ||  BDNF016_RestTremor_butter.xlsx
|| 17 / 23  FINISHED! ||  BDNF017_RestTremor_butter.xlsx
|| 18 / 23  FINISHED! ||  BDNF018_RestTr

## 3a. Calculate Sample Entropy

In [68]:
all_butter = glob.glob(os.path.join(path, "*_butter.xlsx"))  # make a list of paths
shape = len(all_butter)

entropies = {} # initialize dictionary

In [69]:
def calc_vector(row):
    '''
    Calculate the acceleration vector from x,y,z coordinates. Using total acceleration equation.
    '''
    vector = np.sqrt(row['AccelX']**2 + row['AccelY']**2 + row['AccelZ']**2)
    return vector

In [70]:
def sampen_vec_calc(all_butter):
    ''' Calculates the sample entropy for acceleration vectors only. No zscore.
    
    Function to calculate entropy for all files in a list, 
    save it as a dictionary, convert to a dataframe, and format it for easy reading.
    Designed for KinesiaONE tremor raw data
    
    sampen_calc(file_list)
    '''
    i = 0
    output = "samen_tremor_" + prepost + ".xlsx"
    for file in all_butter:
        i = i + 1
        df = pd.read_excel(file)
        file = file.split(sep='\\')[2].split(sep=".")[0] # get the filename
        cols = df.columns
        df['vector'] = df.apply(calc_vector,axis=1) # calculate the acceleration magnitude
        samen = sampen.sampen2(df['vector']) # calculates sample entropy
        entropies[file] = [samen[2][1]]
        print("||", i, '/', shape, " FINISHED! || ", file)
    df_samen = pd.DataFrame.from_dict(entropies) # convert dictionary to dataframe
    df_samen = df_samen.reset_index() # reset index to be normal
    samen = df_samen.pivot_table(columns="index").loc[:,:'RotateZ'] # remove entropy for time, pivot table
    samen.to_excel(output) # save to excel
    print("All entropies saved to ", output)
    return samen

## 3b. Calculate Approximate Entropy using EntroPy

* perm_entropy(x, order=3, normalize=True)                 # Permutation entropy
* spectral_entropy(x, 100, method='welch', normalize=True) # Spectral entropy
* svd_entropy(x, order=3, delay=1, normalize=True)         # Singular value decomposition entropy
* app_entropy(x, order=2, metric='chebyshev')              # Approximate entropy
* sample_entropy(x, order=2, metric='chebyshev')           # Sample entropy
* lziv_complexity('01111000011001', normalize=True)        # Lempel-Ziv complexity


In [71]:
def appen_vec_calc(all_butter):
    ''' Calculates the approximate entropy for acceleration vectors only. No zscore.
    
    Function to calculate entropy for all files in a list, 
    save it as a dictionary, convert to a dataframe, and format it for easy reading.
    Designed for KinesiaONE tremor raw data
    
    appen_calc(file_list)
    '''
    i = 0
    output = "appen_tremor_" + prepost + ".xlsx"
    for file in all_butter:
        i = i + 1
        df = pd.read_excel(file)
        file = file.split(sep='\\')[2].split(sep=".")[0] # get the filename
        cols = df.columns
        df['vector'] = df.apply(calc_vector,axis=1) # calculate the acceleration magnitude
        appen = app_entropy(np.array(df['vector']), order=2, metric='chebyshev') # calculates sample entropy
        entropies[file] = [appen]
        print("||", i, '/', shape, " FINISHED! || ", file)
    df_appen = pd.DataFrame.from_dict(entropies) # convert dictionary to dataframe
    df_appen = df_appen.reset_index() # reset index to be normal
    appen = df_appen.pivot_table(columns="index").loc[:,:'RotateZ'] # remove entropy for time, pivot table
    appen.to_excel(output) # save to excel
    print("All entropies saved to ", output)
    return appen

In [72]:
if entropy == "appen":
    if prepost == "post":
        print("Analyzing ApEn of post data")
        post_ap = appen_vec_calc(all_butter)
    elif prepost =="pre":
        print("Analyzing ApEn of pre data")
        pre_ap = appen_vec_calc(all_butter)
    else:
        print("Error. Check prepost assignment")
elif entropy == "samen":
    if prepost == "post":
        print("Analyzing SamEn of post data")
        post_sam = sampen_vec_calc(all_butter)
    elif prepost =="pre":
        print("Analyzing SamEn of pre data")
        pre_sam = sampen_vec_calc(all_butter)
    else:
        print("Error. Check prepost assignment")

Analyzing SamEn of post data
|| 1 / 20  FINISHED! ||  BDNF001_RestTremor_butter
|| 2 / 20  FINISHED! ||  BDNF002_RestTremor_butter
|| 3 / 20  FINISHED! ||  BDNF003_RestTremor_butter
|| 4 / 20  FINISHED! ||  BDNF004_RestTremor_butter
|| 5 / 20  FINISHED! ||  BDNF005_RestTremor_butter
|| 6 / 20  FINISHED! ||  BDNF006_RestTremor_butter
|| 7 / 20  FINISHED! ||  BDNF007_RestTremor_butter
|| 8 / 20  FINISHED! ||  BDNF008_RestTremor_butter
|| 9 / 20  FINISHED! ||  BDNF009_RestTremor_butter
|| 10 / 20  FINISHED! ||  BDNF010_RestTremor_butter
|| 11 / 20  FINISHED! ||  BDNF012_RestTremor_butter
|| 12 / 20  FINISHED! ||  BDNF013_RestTremor_butter
|| 13 / 20  FINISHED! ||  BDNF014_RestTremor_butter
|| 14 / 20  FINISHED! ||  BDNF015_RestTremor_butter
|| 15 / 20  FINISHED! ||  BDNF017_RestTremor_butter
|| 16 / 20  FINISHED! ||  BDNF020_RestTremor_butter
|| 17 / 20  FINISHED! ||  BDNF021_RestTremor_butter
|| 18 / 20  FINISHED! ||  BDNF022_RestTremor_butter
|| 19 / 20  FINISHED! ||  BDNF023_RestTremor

## 4. Export

### Export AppEn

In [74]:
df_pre = pre_ap.copy()
df_pre = df_pre.reset_index()
df_pre.columns = ['participant','appen_pre']

In [75]:
df_post = post_ap.copy()
df_post = df_post.reset_index()
df_post.columns = ['participant','appen_post']

In [47]:
df_both = pd.merge(df_pre, df_post, on="participant")
df_both['appen_diff'] = df_both['appen_post']-df_both['appen_pre']

In [48]:
df_both

,participant,appen_pre,appen_post,appen_diff
0,BDNF001_RestTremor_butter,0.283562,0.355282,0.071720
1,BDNF002_RestTremor_butter,0.005238,0.283725,0.278487
2,BDNF003_RestTremor_butter,0.034666,0.047962,0.013295
3,BDNF004_RestTremor_butter,0.001696,0.000208,-0.001488
4,BDNF005_RestTremor_butter,0.300445,0.600023,0.299578
5,BDNF006_RestTremor_butter,0.454351,0.000555,-0.453796
6,BDNF007_RestTremor_butter,0.515034,0.460612,-0.054422
7,BDNF008_RestTremor_butter,0.013010,0.084442,0.071432
8,BDNF009_RestTremor_butter,0.254080,0.413733,0.159653
9,BDNF010_RestTremor_butter,0.005020,0.000526,-0.004494


In [49]:
df_kin = pd.read_csv('kinesia_scores.csv')

In [50]:
appen_score = pd.merge(df_both, df_kin, left_on="participant", right_on="BDNF")
appen_score = appen_score.drop(["BDNF","PreFreq","PostFreq"], axis=1)
appen_score["score_diff"] = appen_score['Postscore']-appen_score['Prescore']
appen_score = appen_score.sort_values(by="score_diff",ascending=True)

In [52]:
appen_score.to_excel("tremor_appen.xlsx")

### Export SamEn

In [76]:
df_pre = pre_sam.copy()
df_pre = df_pre.reset_index()
df_pre.columns = ['participant','samen_pre']

In [77]:
df_post = post_sam.copy()
df_post = df_post.reset_index()
df_post.columns = ['participant','samen_post']

In [78]:
df_both = pd.merge(df_pre, df_post, on="participant")
df_both['samen_diff'] = df_both['samen_post']-df_both['samen_pre']

In [79]:
df_kin = pd.read_csv('kinesia_scores.csv')

In [80]:
samen_score = pd.merge(df_both, df_kin, left_on="participant", right_on="BDNF")
samen_score = samen_score.drop(["BDNF","PreFreq","PostFreq"], axis=1)
samen_score["score_diff"] = samen_score['Postscore']-samen_score['Prescore']
samen_score = samen_score.sort_values(by="score_diff",ascending=True)

In [81]:
samen_score

,participant,samen_pre,samen_post,samen_diff,Prescore,Postscore,score_diff
14,BDNF017_RestTremor_butter,0.004166,0.000009,-4.156896e-03,2.3,0.2,-2.1
6,BDNF007_RestTremor_butter,0.221788,0.010782,-2.110062e-01,3.3,2.5,-0.8
17,BDNF022_RestTremor_butter,0.000000,0.000000,0.000000e+00,1.0,0.3,-0.7
5,BDNF006_RestTremor_butter,0.000009,0.000018,9.146323e-06,0.5,0.2,-0.3
19,BDNF024_RestTremor_butter,0.000000,0.000000,0.000000e+00,0.1,0.0,-0.1
13,BDNF015_RestTremor_butter,0.004202,0.000000,-4.201799e-03,0.1,0.0,-0.1
16,BDNF021_RestTremor_butter,0.000000,0.004184,4.183773e-03,0.0,0.0,0.0
15,BDNF020_RestTremor_butter,0.000009,0.000000,-8.882927e-06,0.0,0.0,0.0
12,BDNF014_RestTremor_butter,0.004193,0.000000,-4.192767e-03,0.0,0.0,0.0
10,BDNF012_RestTremor_butter,0.000009,0.000000,-8.958286e-06,0.8,0.8,0.0


In [82]:
samen_score.to_excel("tremor_samen.xlsx")